In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:

train_features = pd.read_csv('https://github.com/Palakjhamnani/Hackathon-IIT-Guwahati/raw/main/training_set_features.csv')
train_labels = pd.read_csv('https://github.com/Palakjhamnani/Hackathon-IIT-Guwahati/raw/main/training_set_labels.csv')
test_features = pd.read_csv('https://github.com/Palakjhamnani/Hackathon-IIT-Guwahati/raw/main/test_set_features.csv')

In [ ]:
train_features = train_features.drop(columns=['respondent_id'])
test_respondent_ids = test_features['respondent_id']
test_features = test_features.drop(columns=['respondent_id'])


In [ ]:
categorical_features = train_features.select_dtypes(include=['object']).columns
numerical_features = train_features.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
X = train_features
y = train_labels.drop(columns=['respondent_id'])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = MultiOutputClassifier(RandomForestClassifier(random_state=42))


In [ ]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

In [ ]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_to...
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa', 'employment_industry', 'employment_occupation'],
      dtype='object'))])),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

In [ ]:
y_val_pred = clf.predict_proba(X_val)

In [ ]:
y_val_pred_xyz = y_val_pred[0][:, 1]
y_val_pred_seasonal = y_val_pred[1][:, 1]

In [ ]:
roc_auc_xyz = roc_auc_score(y_val['xyz_vaccine'], y_val_pred_xyz)
roc_auc_seasonal = roc_auc_score(y_val['seasonal_vaccine'], y_val_pred_seasonal)

In [ ]:
mean_roc_auc = np.mean([roc_auc_xyz, roc_auc_seasonal])

print(f'Mean ROC AUC score: {mean_roc_auc}')

Mean ROC AUC score: 0.8606129501926918


In [ ]:
test_pred = clf.predict_proba(test_features)


In [ ]:
test_pred_xyz = test_pred[0][:, 1]
test_pred_seasonal = test_pred[1][:, 1]

In [ ]:
submission = pd.DataFrame({
    'respondent_id': test_respondent_ids,
    'xyz_vaccine': test_pred_xyz,
    'seasonal_vaccine': test_pred_seasonal
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>